# Validation tests for Sprinkler

This notebook is not for an application of DC2 data but validation tests for Sprinkler, which is a module for embedding strong lensing systems in DC2. 

### Done
- Compared the statistical properties of strong lensing systems created by Sprinkler with the predictions in Oguri & Marshall 2010 (OM10).
- Found inconsistencies due to the luminosity and redshift distributions of AGNs in DC2.
- Involved weighted (magnitudes and redshifts) matching methods to correct the inconsistencies.

### ToDo
- As is shown in the latest figure below, the corrections are noticeable but not perfect, we will keep trying other matching strategies to match the results from Sprinker to the predictions from OM10 as well as possible.

***The kernel of this notebook is still Python2 because the OM10 Catalog used here is in Python2.***

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib as mpl
mpl.use('Agg')

import matplotlib.pyplot as plt
import sys
sys.path.append("/global/u2/n/nlidesc/.local/lib/python3.6/site-packages/")

import numpy as np
import astropy.io.fits as pyfits
import pandas as pd
import om10
from om10 import db

import seaborn as sns;sns.set()
import warnings;warnings.filterwarnings('ignore')

data_dir = "/global/projecta/projectdirs/lsst/groups/SL/users/nanli/Projects/sprinkler_validation_SprintWeek/data/"

In [ ]:
#--------------------------------------------
# Tests for reading in OM10 lenses
#

data_test = db.DB(data_dir + "qso_mock.fits")
# data_test.select_random(maglim=99.0,area=18000.0,IQ=0.7)
data_test.select_random()
good = data_test.sample
mags_srcs = good['MAGI_IN']
zred_srcs = good['ZSRC']

idx1 = zred_srcs >=1.9
idx2 = zred_srcs < 2.1

idx = idx1&idx2

print len(mags_srcs[idx])

out_mags_data = mags_srcs[idx]

out_mags_data.astype(np.float32).tofile('magi_in_2818.bin')

# data_test.select_random(Nlens=1000)
# good = data_test.sample

In [ ]:
#-----------------------
# OM10
#

def load_in_OM10():
    ddb = db.DB(data_dir + "qso_mock.fits")
    ddb.select_random(maglim=99.0,area=18000.0,IQ=0.7)
    good = ddb.sample[np.where(ddb.sample['ZSRC'] < 3.6)]
    print "Number in cosmography sub-sample:",len(good)

    data_om10 = {}
    data_om10['zlens'] = good['ZLENS']
    data_om10['zsrcs'] = good['ZSRC']
    data_om10['tdmax'] = np.log10(np.max(good['DELAY'], axis=1)) #/30.0 #month
    data_om10['spmax'] = good['IMSEP']
    data_om10['apmdi'] = good['APMAG_I']
    data_om10['apmsi'] = good['MAGI_IN']
    data_om10['apmli'] = good['MAGI_IN'] - 2.5*np.log10(np.sum(np.abs(good['MAG']), axis=1))
    data_om10['Datasets'] = 'OM10'

    df_om10 = pd.DataFrame(data=data_om10)
    return df_om10

In [ ]:
#-----------------------
# MatchedAGNs
#

def load_in_MatchedAGNs():
    ugly = pyfits.getdata(data_dir + "cosmoDC2_v1.0_matched_AGN.fits")

    data_MatchedAGNs = {}
    data_MatchedAGNs['zlens'] = ugly['ZLENS']
    data_MatchedAGNs['zsrcs'] = ugly['ZSRC']
    data_MatchedAGNs['tdmax'] = np.log10(np.max(ugly['DELAY'], axis=1)) #/30.0 #month
    data_MatchedAGNs['spmax'] = ugly['IMSEP']
    data_MatchedAGNs['apmdi'] = ugly['APMAG_I']
    data_MatchedAGNs['apmsi'] = ugly['MAGI_IN']
    data_MatchedAGNs['apmli'] = ugly['MAGI_IN'] - 2.5*np.log10(np.sum(np.abs(ugly['MAG']), axis=1))
    data_MatchedAGNs['Datasets'] = 'MatchedAGNs'

    df_MatchedAGNs = pd.DataFrame(data=data_MatchedAGNs)
    return df_MatchedAGNs

In [ ]:
dataf_good   = load_in_OM10()
dataf_ugly   = load_in_MatchedAGNs()

In [ ]:
%%time

frames = [dataf_good, dataf_ugly]
result = pd.concat(frames)

pal = dict(OM10=sns.color_palette()[0], 
           MatchedAGNs=sns.color_palette()[2],
           NewMatchedAGNs=sns.color_palette()[1])

pg = sns.PairGrid(result, vars=('zlens', 'zsrcs', 'tdmax', 'spmax', 'apmdi', 'apmsi', 'apmli'), hue='Datasets', palette=pal,
                  hue_kws={"cmap": ["Blues", "Reds", "Greens"]})
pg.map_diag(sns.kdeplot)
pg.map_upper(sns.kdeplot, n_levels=3)
pg.map_lower(plt.scatter, alpha=0.1)
pg.add_legend();
del frames
del result

In [ ]:
def load_in_NewMatchedAGNs():
    ugly = pyfits.getdata(data_dir + "cosmoDC2_v1.0_matched_AGN_redshift_mag_weighted.fits")

    data_NewMatchedAGNs = {}
    data_NewMatchedAGNs['zlens'] = ugly['ZLENS']
    data_NewMatchedAGNs['zsrcs'] = ugly['ZSRC']
    data_NewMatchedAGNs['tdmax'] = np.log10(np.max(ugly['DELAY'], axis=1)) #/30.0 #month
    data_NewMatchedAGNs['spmax'] = ugly['IMSEP']
    data_NewMatchedAGNs['apmdi'] = ugly['APMAG_I']
    data_NewMatchedAGNs['apmsi'] = ugly['MAGI_IN']
    data_NewMatchedAGNs['apmli'] = ugly['MAGI_IN'] - 2.5*np.log10(np.sum(np.abs(ugly['MAG']), axis=1))
    data_NewMatchedAGNs['Datasets'] = 'NewMatchedAGNs'

    df_NewMatchedAGNs = pd.DataFrame(data=data_NewMatchedAGNs)
    return df_NewMatchedAGNs

In [ ]:
dataf_pretty = load_in_NewMatchedAGNs()

In [ ]:
%%time

frames = [dataf_good, dataf_ugly, dataf_pretty]
result = pd.concat(frames)

pal = dict(OM10=sns.color_palette()[0], 
           MatchedAGNs=sns.color_palette()[2],
           NewMatchedAGNs=sns.color_palette()[1])

pg = sns.PairGrid(result, vars=('zlens', 'zsrcs'), hue='Datasets', hue_kws={"cmap": ["Blues", "Reds", "Greens"]}, palette=pal)
pg.fig.set_size_inches(11,10)
pg.map_diag(sns.distplot, kde_kws={"lw": 3})
pg.map_upper(sns.kdeplot,lw=3, n_levels=3)
pg.map_lower(plt.scatter, alpha=0.5)
pg.add_legend();

In [ ]:
pg = sns.PairGrid(result, vars=('apmsi', 'apmli'), hue='Datasets', hue_kws={"cmap": ["Blues", "Greens", "Reds"]}, palette=pal)
pg.fig.set_size_inches(11,10)
pg.map_diag(sns.distplot, kde_kws={"lw": 3})
pg.map_upper(sns.kdeplot, lw=3, n_levels=3)
pg.map_lower(plt.scatter, alpha=0.5)
pg.add_legend();